In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.optim as optim
from script.tools import tools
from script.collection import Parameter as pmt
import sys
print(sys.executable)
import net
import os
from script.tools import collection
result = collection.collection("collection/workspace1")  #[iops,dic_ssd,dic_worload,f.path]
result = collection.collection("collection/workspace2",result)
plst = collection.getParameters(path="collection/config.xlsx",id = "1")



/opt/homebrew/anaconda3/bin/python
collection:  472
collection:  1774


/opt/homebrew/anaconda3/lib/python3.9/site-packages/openpyxl/reader/excel.py:237: UserWarning: Data Validation extension is not supported and will be removed
  ws_parser.bind_all()


In [2]:
xlst = []
ylst = []
xylst = []
keys = []
expect = ['Seed','key','default']
for iops,dic_ssd,dic_worload,path in result:
    dic = dic_ssd
    dic.update(dic_worload)
    values,keys = collection.getUsefullKeys(dic,plst,expect)
    if iops == 'inf':
        #print("inf------")
        continue
    xlst.append(values)
    ylst.append([float(iops)])
    xylst.append([values,[float(iops)]])
    keys = keys
print(len(xlst)," ",len(ylst))
print(len(xlst[0]))
#print(keys)
#print(ylst)

1714   1714
48


In [3]:
x_tensor = torch.tensor(xlst,dtype=torch.float)
y_tensor = torch.tensor(ylst,dtype=torch.float)
print(x_tensor.size(),y_tensor.size())
tensor1 = torch.tensor([[1, 2, 3],
                       [4, 5, 6]])
print(tensor1.size())
#x_ytensor = torch.tensor(xylst,dtype=torch.float)


# 计算均值和标准差
x_mean = x_tensor.mean(dim=0)
x_std = x_tensor.std(dim=0)
y_mean = y_tensor.mean(dim=0)
y_std = y_tensor.std(dim=0)

# 归一化 x 和 y
normalized_x = (x_tensor - x_mean) / x_std
normalized_y = (y_tensor - y_mean) / y_std
normalized_y_std = y_tensor  / y_std
#print("x_mean: ",x_mean)
#print("x_std: ",x_std)
#print("y_mean: ",y_mean)
#print("y_std: ",y_std)
#print(x_tensor[0])
#print(keys)

tree_ssd,rootssd = tools.getTree("collection/ssdconfig.xml")
tree_workload,root_workload = tools.getTree("collection/workload.xml")
i = 0
for key in keys:
    default = tools.getext(rootssd,key)
    if default == None:
        default = tools.getext(root_workload,key)
    print(key,"default: ", default," mean:",str(x_mean[i].item())," std：",x_std[i].item())
    i = i + 1


torch.Size([1714, 48]) torch.Size([1714, 1])
torch.Size([2, 3])
PCIe_Lane_Bandwidth default:  1.00000  mean: 2.0686001777648926  std： 2.269704580307007
PCIe_Lane_Count default:  4  mean: 19.761960983276367  std： 11.245830535888672
SATA_Processing_Delay default:  400000  mean: 2025643.125  std： 1128192.875
Queue_Fetch_Size default:  512  mean: 2582.690673828125  std： 1467.95458984375
Data_Cache_DRAM_Row_Size default:  8192  mean: 40766.30078125  std： 23435.74609375
Data_Cache_DRAM_Data_Rate default:  100  mean: 503.1376953125  std： 285.00799560546875
Data_Cache_DRAM_Data_Busrt_Size default:  1  mean: 5.575262546539307  std： 2.9104950428009033
Data_Cache_DRAM_tRCD default:  13  mean: 65.455078125  std： 37.691986083984375
Data_Cache_DRAM_tCL default:  13  mean: 64.97899627685547  std： 37.967872619628906
Data_Cache_DRAM_tRP default:  13  mean: 64.94457244873047  std： 37.416290283203125
Overprovisioning_Ratio default:  0.07  mean: 0.18851685523986816  std： 0.1502837985754013
GC_Exec_Thresho

In [9]:
model = net.DeepNet(48,48,1,5)
#device = torch.device("mps")
#device = torch.device("cpu")
#model.to(device)
criterion = nn.MSELoss()

In [26]:
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
# 自定义数据集类
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# dataset = CustomDataset(x_tensor, y_tensor)
y_modified = y_tensor/y_mean
dataset = CustomDataset(normalized_x, y_modified)
batch_size = 5
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
#optimizer = optim.SGD(model.parameters(), lr=0.001)




In [23]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 1000

loss_history = []
for epoch in range(num_epochs):
    total_loss = 0.0
    i = 0
    for batch_x, batch_y in dataloader:
        #print("hhhh ")
        #print(batch_x,batch_x)
        #i = i +1
        #print(batch_x,"\n",batch_y)
        #print("zzzzzzz")
        # 前向传播

        outputs = model(batch_x)
        #print("batch_x",batch_x[0],"\n outputs: ",outputs[0])
        loss = criterion(outputs, batch_y)
        i = i +1
        if i == -1:
            print("i=------"+str(i),"\n outputs: ",outputs,"\n y:",batch_y)
        total_loss += loss.item()
        if False:
            print("   \n \n")
            print(" outputs: ", outputs)
            print(" batch_y", batch_y)
            print(" loss ", loss)

            print("   \n \n")
        #print(f'Epoch [{epoch+1}/{num_epochs}]')
        #print("loss:",loss.item(),"batch_x: ",batch_x," outputs ",outputs," batch_y: ",batch_y)
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #loss_history.append(total_loss)
    # 打印训练信息
    if (epoch+1) % 10 == 0:
        average_loss = total_loss / len(dataloader)
        #print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}')



Epoch [10/1000], Loss: 0.0038
Epoch [20/1000], Loss: 0.0055
Epoch [30/1000], Loss: 0.0047
Epoch [40/1000], Loss: 0.0040
Epoch [50/1000], Loss: 0.0048
Epoch [60/1000], Loss: 0.0022
Epoch [70/1000], Loss: 0.0024
Epoch [80/1000], Loss: 0.0037
Epoch [90/1000], Loss: 0.0061
Epoch [100/1000], Loss: 0.0019
Epoch [110/1000], Loss: 0.0052
Epoch [120/1000], Loss: 0.0071
Epoch [130/1000], Loss: 0.0066
Epoch [140/1000], Loss: 0.0034
Epoch [150/1000], Loss: 0.0036
Epoch [160/1000], Loss: 0.0035
Epoch [170/1000], Loss: 0.0031
Epoch [180/1000], Loss: 0.0025
Epoch [190/1000], Loss: 0.0041
Epoch [200/1000], Loss: 0.0036
Epoch [210/1000], Loss: 0.0036
Epoch [220/1000], Loss: 0.0081
Epoch [230/1000], Loss: 0.0050
Epoch [240/1000], Loss: 0.0031
Epoch [250/1000], Loss: 0.0043
Epoch [260/1000], Loss: 0.0031
Epoch [270/1000], Loss: 0.0044
Epoch [280/1000], Loss: 0.0021
Epoch [290/1000], Loss: 0.0040
Epoch [300/1000], Loss: 0.0036
Epoch [310/1000], Loss: 0.0016
Epoch [320/1000], Loss: 0.0024
Epoch [330/1000],

In [28]:
import numpy as np
y_predict_train = model(normalized_x)
y_err = (y_predict_train - y_modified)/y_modified
print(torch.sum(abs(y_err))/len(y_predict))

def test(x_,y_,m,rate_y, mean_x = 0,std_x = 1):
    x_ = (x_ - mean_x)/std_x
    y_pre = m(x_)*rate_y
    y_e = abs(y_pre - y_)/y_
    print(torch.sum(y_e)/len(y_))
# 训练集
test(x_tensor,y_tensor,model,y_mean,x_mean,x_std);
# 测试集



tensor(0.1561, grad_fn=<DivBackward0>)
1111111
tensor(0.1561, grad_fn=<DivBackward0>)


In [16]:
result1 = []
result1 = collection.collection("collection/workspace",result1)

collection:  1070


In [48]:
#print(xlst[0])
print(len(xlst[0]))
tx = []
tx.append(keys)
i = 0
for y_ in ylst:
    x_ = xlst[i].copy()
    x_.append(y_[0])
    tx.append(x_)
    i = i + 1
print(tx[0])
tools.lst2excel(tx,"test.xlsx")

48
['PCIe_Lane_Bandwidth', 'PCIe_Lane_Count', 'SATA_Processing_Delay', 'Queue_Fetch_Size', 'Data_Cache_DRAM_Row_Size', 'Data_Cache_DRAM_Data_Rate', 'Data_Cache_DRAM_Data_Busrt_Size', 'Data_Cache_DRAM_tRCD', 'Data_Cache_DRAM_tCL', 'Data_Cache_DRAM_tRP', 'Overprovisioning_Ratio', 'GC_Exec_Threshold', 'Preferred_suspend_erase_time_for_read', 'Preferred_suspend_erase_time_for_write', 'Preferred_suspend_write_time_for_read', 'Flash_Channel_Count', 'Flash_Channel_Width', 'Channel_Transfer_Rate', 'Chip_No_Per_Channel', 'Page_Read_Latency_LSB', 'Page_Read_Latency_CSB', 'Page_Read_Latency_MSB', 'Page_Program_Latency_LSB', 'Page_Program_Latency_CSB', 'Page_Program_Latency_MSB', 'Block_Erase_Latency', 'Block_PE_Cycles_Limit', 'Suspend_Erase_Time', 'Suspend_Program_Time', 'Die_No_Per_Chip', 'Plane_No_Per_Die', 'Block_No_Per_Plane', 'Page_No_Per_Block', 'Page_Capacity', 'Page_Metadat_Capacity', 'Channel_IDs', 'Chip_IDs', 'Die_IDs', 'Plane_IDs', 'Initial_Occupancy_Percentage', 'Working_Set_Percentag

In [135]:
print(len(keys))
import platform

print("Python版本:", platform.python_version())

48
Python版本: 3.9.13
